# SQL

In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd

In [2]:
engine = create_engine('sqlite:////content/test.db')
connection = engine.connect()
def select(sql):
  return pd.read_sql(text(sql),connection)

Напишите запрос, который выведет, сколько времени в среднем задачи каждой группы
находятся в статусе “Open”
Условия:
Под группой подразумевается первый символ в ключе задачи. Например, для ключа
“C-40460” группой будет “C”
Задача может переходить в один и тот же статус несколько раз.
Переведите время в часы с округлением до двух знаков после запятой.


In [ ]:
sql = '''
SELECT SUBSTR(issue_key, 1, 1) AS group_key,
       ROUND(AVG(minutes_in_status / 60.0), 2) AS avg_open_hours
FROM history
WHERE status = 'Open'
GROUP BY group_key
'''
select(sql)

,group_key,avg_open_hours
0,A,153.69
1,B,191.94
2,C,308.20
3,D,570.12
4,E,405.62


Напишите запрос, который выведет ключ задачи, последний статус и его время создания
для задач, которые открыты на данный момент времени.
Условия:

Открытыми считаются задачи, у которых последний статус в момент времени не “Closed” и
не “Resolved”
Задача может переходить в один и тот же статус несколько раз.

Оформите запрос таким образом, чтобы, изменив дату, его можно было использовать для
поиска открытых задач в любой момент времени в прошлом
Переведите время в текстовое представление

In [100]:
sql = '''
SELECT issue_key, previous_status, prev_creation_time
FROM (
  SELECT issue_key, previous_status,
         LAG(DATE(started_at / 1000, 'unixepoch'), 1, DATE(started_at / 1000, 'unixepoch')) OVER (PARTITION BY issue_key ORDER BY started_at) AS prev_creation_time,
         ROW_NUMBER() OVER (PARTITION BY issue_key ORDER BY started_at DESC) AS rn
  FROM history
  WHERE previous_status NOT IN ('Closed', 'Resolved')
    AND started_at <= CAST(strftime('%s', '2022-09-31') AS INT) * 1000 -- change date
) AS subquery
WHERE rn = 1

ORDER BY prev_creation_time;
  '''
select(sql)

,issue_key,previous_status,prev_creation_time
0,C-1,Open,2019-06-04
1,A-22950,Waiting for integration,2020-04-14
2,A-23187,Waiting for integration,2020-04-14
3,A-23381,Waiting for integration,2020-04-17
4,A-23382,Waiting for integration,2020-04-17
...,...,...,...
1267,A-43989,In Progress,2022-09-28
1268,A-46662,In Review,2022-09-28
1269,A-50015,Waiting for integration,2022-09-28
1270,D-6403,Open,2022-09-28


# Python

Сообщения в заданных файлах упорядочены по полю timestamp в порядке возрастания.
Требуется написать скрипт, который объединит эти два файла в один.
При этом сообщения в получившемся файле тоже должны быть упорядочены в порядке
возрастания по полю timestamp.

К заданию прилагается вспомогательный скрипт на python3, который создает два файла
"log_a.jsonl" и "log_b.jsonl".
Командлайн для запуска:
log_generator.py <path/to/dir>
Ваше приложение должно поддерживать следующий командлайн:
<your_script>.py <path/to/log1> <path/to/log2> -o <path/to/merged/log>

In [101]:
%%writefile log_generator.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python


import json
import time
import argparse
import dataclasses
import random
import shutil
from datetime import datetime, timedelta
from pathlib import Path


_MAX_LOG_SIZE_BYTES = 2 ** 30 # 1GB
_LOG_FILENAMES = 'log_a.jsonl', 'log_b.jsonl'
_LOG_LEVELS = b'DEBUG', b'INFO', b'WARNING', b'ERROR'
_PERSON_NAME = 'Bender', 'Fry', 'Leela', 'Amy', 'Farnsworth', 'Dr.Zoidberg'
_ACTION = 'said', 'took', 'played', 'ate', 'saw', 'built', 'killed','created', 'brought', 'robbed'
_OBJECT = 'an apple', 'a car', 'a boat', 'a rocket', 'a mall', 'a fish', 'abottle of bear', 'a man'
_PLACE = 'at park', 'on the Mars', 'near the Square Garden', 'in L.A.'
_WHEN = 'day before yesterday', 'yesterday', 'today', 'tomorrow', 'day after tomorrow'

@dataclasses.dataclass
class LogRecord:
  log_level: str
  timestamp: str
  message: str

def _parse_args() -> argparse.Namespace:
  parser = argparse.ArgumentParser(description='Tool to generate testlogs.')
  parser.add_argument('output_dir',
                      metavar='<OUTPUT DIR>',
                      type=str,
                      help='path to dir with generated logs',)
  parser.add_argument('-f', '--force',
                      action='store_const',
                      const=True,
                      default=False,
                      help='force write logs',
                      dest='force_write',)
  return parser.parse_args()

def _create_dir(dir_path: Path, *, force_write: bool = False) -> None:
  if dir_path.exists():
    if not force_write:
      raise FileExistsError(f'Dir "{dir_path}" already exists. Remove it first orchoose another one.')
      shutil.rmtree(dir_path)

  dir_path.mkdir(parents=True)


_RECORD_TEMPLATE = LogRecord(
    log_level='<LOG_LEVEL>',
    timestamp='<TIMESTAMP>',
    message='<MESSAGE>',
)
_MESSAGE_TEMPLATE = json.dumps(dataclasses.asdict(_RECORD_TEMPLATE)).encode('utf-8')
_MESSAGE_TEMPLATE += b'\n'


def _generate_logfile(log_filepath: Path, start_time: datetime) -> None:
  print(f"generating {log_filepath.name}...")
  person_name, action = _PERSON_NAME, _ACTION
  object, place, when = _OBJECT, _PLACE, _WHEN
  log_levels, message_template = _LOG_LEVELS, _MESSAGE_TEMPLATE
  rand, td, ln = random.random, timedelta, len

  with log_filepath.open('wb') as fh:
    current_time = start_time
    total_size, max_size = 0, _MAX_LOG_SIZE_BYTES
    write = fh.write
    while total_size < max_size:
      timestamp = f"{current_time.year}-{current_time.month:02}-{current_time.day:02} " \
            f"{current_time.hour}:{current_time.minute:02}:{current_time.second:02}".encode('utf-8')
      message = f"{person_name[int(6 * rand())]} " \
                f"{action[int(10 * rand())]} " \
                f"{object[int(8 * rand())]} " \
                f"{place[int(4 * rand())]} " \
                f"{when[int(5 * rand())]}".encode('utf-8')

      data = message_template \
          .replace(b'<LOG_LEVEL>', log_levels[int(4 * rand())]) \
          .replace(b'<TIMESTAMP>', timestamp) \
          .replace(b'<MESSAGE>', message)

      write(data)
      total_size += ln(data)
      current_time += td(seconds=int(10 * rand()))


def _generate_logs(output_dir: Path) -> None:
  start_time = datetime.now()

  for log_filename in _LOG_FILENAMES:
    log_path = output_dir.joinpath(log_filename)
    _generate_logfile(log_path, start_time)


def main() -> None:
  args = _parse_args()
  t0 = time.time()
  output_dir = Path(args.output_dir)
  _create_dir(output_dir, force_write=args.force_write)
  _generate_logs(output_dir)
  print(f"finished in {time.time() - t0:0f} sec")


if __name__ == '__main__':
  main()

Writing log_generator.py


In [102]:
!python3 log_generator.py /content/test

generating log_a.jsonl...
generating log_b.jsonl...
finished in 110.177036 sec


In [116]:
%%writefile py_test.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python

import json
from json import JSONEncoder
import datetime
import time
import argparse
import dataclasses
from dataclasses import dataclass, asdict, field
from typing import Any, List


@dataclass(order=True, slots=True)
class Json_slots:
    sort_index: float = field(init=False, repr=False, compare=True)
    log_level: str
    timestamp: str
    message: str

    @property
    def to_dict(self):
        return {
            'log_level': self.log_level,
            'timestamp': self.timestamp,
            'message': self.message
        }
    # create unix time for sort by date
    def __post_init__(self):
      element = datetime.datetime.strptime(self.timestamp, '%Y-%m-%d %H:%M:%S')
      unix_timestamp = datetime.datetime.timestamp(element)
      self.sort_index = unix_timestamp


# subclass JSONEncoder
class Json_slots_Encoder(JSONEncoder) -> dict:
        def default(self, o):
            return o.to_dict


def get_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser(description='Parser input arguments')
    parser.add_argument('path_1', default='/content/test/log_a.jsonl', type=str,
                        help="Path to log_a")
    parser.add_argument('path_2', default='/content/test/log_b.jsonl', type=str,
                        help="Path to log_b")
    parser.add_argument('-o', '--merged', default='/content', type=str, action="store",
                        help="Path to merged file")

    return parser.parse_args()


def merge_JsonFiles(filename) -> list:
    res_list = []
    for f1 in filename:
      with open(f1, 'r') as infile:
        for line in infile:
          data = json.loads(line)
          log_level = data['log_level']
          timestamp = data['timestamp']
          message = data['message']
          res_list.append(Json_slots(log_level, timestamp, message))

    res_list = sorted(res_list)
    return res_list


def main() -> None:
    args = get_args() # parse args
    files=[]
    files.extend([args.path_1, args.path_2]) # list of the paths to files .jsonl
    to_json = merge_JsonFiles(files)

    # write output result file
    with open(args.merged, 'w') as output_file:
      write = output_file.write
      for line in to_json:
        write(json.dumps(line, cls=Json_slots_Encoder) + '\n')


if __name__ == '__main__':
    main()


Overwriting py_test.py


In [106]:
!python3 py_test.py /content/test/log_a.jsonl /content/test/log_b.jsonl -o /content/test/result.jsonl

In [115]:
lines_number = 10
i=0
with open('/content/test/result.jsonl', 'r') as result:
  for line in result:
    data = json.loads(line)
    print(data)
    i+=1
    if i == lines_number:
      break

{'log_level': 'ERROR', 'timestamp': '2023-07-22 10:39:26', 'message': 'Dr.Zoidberg brought a rocket at park day after tomorrow'}
{'log_level': 'INFO', 'timestamp': '2023-07-22 10:39:26', 'message': 'Bender ate a fish in L.A. yesterday'}
{'log_level': 'ERROR', 'timestamp': '2023-07-22 10:39:27', 'message': 'Farnsworth killed a mall at park yesterday'}
{'log_level': 'INFO', 'timestamp': '2023-07-22 10:39:31', 'message': 'Fry killed a rocket on the Mars day before yesterday'}
{'log_level': 'WARNING', 'timestamp': '2023-07-22 10:39:31', 'message': 'Dr.Zoidberg killed a rocket on the Mars today'}
{'log_level': 'INFO', 'timestamp': '2023-07-22 10:39:33', 'message': 'Farnsworth robbed a boat on the Mars day before yesterday'}
{'log_level': 'DEBUG', 'timestamp': '2023-07-22 10:39:38', 'message': 'Fry robbed a boat at park today'}
{'log_level': 'ERROR', 'timestamp': '2023-07-22 10:39:39', 'message': 'Bender saw a car near the Square Garden yesterday'}
{'log_level': 'DEBUG', 'timestamp': '2023-0